In [1]:
import pandas as pd
import numpy as np

## Parsing Weights & impacts

In [2]:
def parse_weights(df,weights,impacts):
    if len(impacts.split(',')) != df.shape[1]-1 or len(weights.split(',')) != df.shape[1]-1:
        raise ValueError("Number of impacts and weights must match the number of columns in the DataFrame.")

    w = [float(weight) for weight in weights.split(",")]
    imp = [1 if impact == '+' else 0 for impact in impacts.split(",")]
    
    return w,imp


## Making Normalized Decesion Matrix

In [3]:
def normalize_dec_mat(df,w):
    rsumsq=0
    for i,col in enumerate(df.columns[1:]):
        rsumsq=((df[col]**2).sum())**0.5
        df.loc[:df.shape[0] - 1, col] /= rsumsq/w[i]
    return df

## Ideal Vpos & Vneg of each column

In [4]:
def calculate_ideal_solutions(norm_df, impacts):
    vpos = [] 
    vneg = [] 

    for i, col in enumerate(norm_df.columns[1:]):
        if impacts[i] == 1: 
            vpos.append(norm_df[col].max())
            vneg.append(norm_df[col].min())
        else:
            vpos.append(norm_df[col].min())
            vneg.append(norm_df[col].max())

    return vpos, vneg

## Calculate Topsis Score

In [5]:
def calculate_topsis_scores(norm_df, vpos, vneg):
    spos = []
    sneg = []

    for i in range(len(norm_df)):
        spos.append(sum((norm_df.iloc[i, 1:] - vpos) ** 2) ** 0.5)
        sneg.append(sum((norm_df.iloc[i, 1:] - vneg) ** 2) ** 0.5)

    scores = [sneg[i] / (sneg[i] + spos[i]) for i in range(len(spos))]
    return scores

## TOPSIS

In [6]:
def topsis(df,weights,impacts):
    w,imp=parse_weights(df,weights,impacts)
    df=normalize_dec_mat(df,w)
    vpos,vneg=calculate_ideal_solutions(df,imp)
    scores = calculate_topsis_scores(df, vpos, vneg)
    df['Topsis Score'] = scores
    df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)
    return df
    

In [7]:
df = pd.read_excel('102203508-data.xlsx')
weights = "0.25,0.25,0.25,0.25,0.25"
impacts = "-,+,+,+,+"
df=topsis(df, weights, impacts)
df.to_csv('102203508-result.csv')